In [ ]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim import Adam
import torch.nn.functional as F

import csv
from skimage import io

from PIL import Image
import pandas as pd

import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

import matplotlib.pyplot as plt
import time
import os
import copy

import import_ipynb
import ResNetCaps

verbose = False
USE_CUDA = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

MNIST_USE = False
CIFAR10_USE = True
MARVEL_USE = False

#matrix product for bilinear function
euclidean = True
kronecker = False
outer_m = False

In [ ]:
class MARVEL_dataset(Dataset):
    def __init__(self, dat_file,train = True, transform = None):   
        self.root_dir = os.path.dirname(dat_file)
        datContent = [i.strip().split(',') for i in open(dat_file).readlines()]
        if train:
            csv_file = os.path.join(self.root_dir, "data_Train.csv")
        else:
            csv_file = os.path.join(self.root_dir, "data_Test.csv")
        with open(csv_file, "w") as f:
            writer = csv.writer(f,delimiter=',')
            writer.writerow(["counter", "set", "class", "label","location"])
            for line in datContent:
                if train and line[1]=='1':
                    if not(line[4] == '-'):
                        writer.writerows([line])  
                if not(train) and line[1] == '2':
                    if not(line[4]=='-'):
                        writer.writerows([line]) 
                
        self.MARVEL_datafile = pd.read_csv(csv_file)       
        self.transform = transform
        
    def __len__(self):
        return len(self.MARVEL_datafile)
    
    def __getitem__(self,idx):
        img_name = self.MARVEL_datafile.iloc[idx,4]
        image = self.__loadfile(img_name)
        target = self.MARVEL_datafile.iloc[idx,2]
        if self.transform:
            image = Image.fromarray(image)
            sample = self.transform(image)
        else:
            sample = image
        return (sample,target)
    
    def __loadfile(self, data_file):
        image = io.imread(data_file)
        if len(image.shape)<3:
            image = np.stack((image,)*3, axis=-1)
        return image

In [ ]:
dataset_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),        
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])


batch_size = 32


if CIFAR10_USE: 
    NUM_CLASSES = 10
    print("CIFAR10")
    image_datasets = {'train': datasets.CIFAR10('../data', train=True, download=True, transform=dataset_transform),'val': datasets.CIFAR10('../data', train=False, download=True, transform=dataset_transform)}
    print("Initializing Datasets and Dataloaders...")

    dataloaders = {'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True) , 'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=True) }
    print("Initializing Datasets and Dataloaders...")

if MARVEL_USE: 
    NUM_CLASSES = 26
    print("MARVEL")
    dat_file = "/home/rita/JupyterProjects/EYE-SEA/DataSets/marveldataset2016-master/FINAL.dat"

    image_datasets = {'train': MARVEL_dataset(dat_file,train = True,transform=dataset_transform),'val': MARVEL_dataset(dat_file,train = False,transform=dataset_transform)}
    print("Initializing Datasets and Dataloaders...")

    dataloaders = {'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True) , 'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=True) }
    print("Initializing Datasets and Dataloaders...")

if MNIST_USE: 
    dataset_transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),        
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
    ])
    NUM_CLASSES = 10
    print("MNIST")
    image_datasets = {'train': datasets.MNIST('../data', train=True, download=True, transform=dataset_transform),'val': datasets.MNIST('../data', train=False, download=True, transform=dataset_transform)}
    print("Initializing Datasets and Dataloaders...")

    dataloaders = {'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True) , 'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=True) }
    print("Initializing Datasets and Dataloaders...")

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [ ]:
class Bilinear(nn.Module):
    def __init__(self,NUM_CLASSES):
        super(Bilinear, self).__init__()
        #Features function :
        self.modelCaps1 = ResNetCaps.ResNetCaps(NUM_CLASSES)
        self.modelCaps2 = ResNetCaps.ResNetCaps(NUM_CLASSES)
        #Classification function:
        self.modelLin = nn.Linear(10, NUM_CLASSES)
        
    def forward(self,inputs):
        #1) Extract features functions vectors (I need to do MATRIX OUTER PRODUCT)
        digit1, masked = self.modelCaps1(inputs)
        digit2, masked = self.modelCaps2(inputs)
        #2) Classification Function    
        output = F.softmax(self.modelLin(self.bilinear(digit1, digit2)),dim=1)
        
        return output, masked
 
    def bilinear(self, A, B):
        self.A = A
        self.B = B
        
        Z_list = []
        labels_size = A.size()[1]
        batch = min(A.size()[0],batch_size)
        for i in range(batch):
        
            a = A[i,:,:,0]
            b = B[i,:,:,0]
            #1.1) Pooling for aggregation of features vectors 
            if euclidean:
                #EUCLIDEAN MATRIX PRODUCT
                if verbose: print("Dim A {} B {}".format(a.shape,b.shape))
                x = torch.mm(a,torch.transpose(b,0,1))
                x = torch.sum(x, dim=1)
            if outer_m:
                #OUTER MATRIX PRODUCT
                A = torch.sum(A,dim=1)
                B = torch.sum(B,dim=1)
                x = torch.ger(A,B)
                x = torch.sum(x, dim=1)
            if kronecker:
                #KRONECKER MATRIX PRODUCT
                x = torch.kron(A.cpu().numpy(),B.cpu().numpy())
                x = torch.from_numpy(x).float().to(device)
                
            y = torch.sign(x)*torch.sqrt(torch.FloatTensor.abs_(x))
            z = y/torch.norm(y)       
            
            Z_list.append(z)
            
        Z = torch.cat(Z_list,0)
        Z = Z.view([batch,labels_size])
        return Z    

In [ ]:
#Load model
model = Bilinear(10)
model = model.to(device)

#optimizers
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),lr = 0.001)
n_epochs = 10
#train
start = time.time()
#batch_id = 100
#inputs, labels = next(iter(dataloaders['train']))
accuracy_train = []
loss_train = []


for epoch in range(n_epochs): 
    model.train() 
    train_loss = 0
    train_accuracy = 0

    print('epoch {}:{}'.format(epoch+1, n_epochs)) 
    for batch_id, (inputs, labels) in enumerate(dataloaders['train']):

        labels =torch.eye(NUM_CLASSES).index_select(dim=0, index=labels)
        inputs, labels = Variable(inputs), Variable(labels)
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs, masked = model(inputs)
        if verbose: print(outputs)
        if verbose: print(labels.long())
        _,label = torch.max(labels, 1)
        if verbose: print(label)
        loss = criterion(outputs, label.long())
        loss.backward()
        optimizer.step()

        train_loss += loss.data[0]
        train_accuracy += (sum(np.argmax(outputs.data.cpu().numpy(), 1) == np.argmax(labels.data.cpu().numpy(), 1)) / float(batch_size))

        if batch_id % 100 == 0:
            print("train accuracy:", sum(np.argmax(outputs.data.cpu().numpy(), 1) == 
                                   np.argmax(labels.data.cpu().numpy(), 1)) / float(batch_size))
            if verbose: print("masked {}".format(np.argmax(masked.data.cpu().numpy(), 1)))
            if verbose: print("labels {}".format(np.argmax(labels.data.cpu().numpy(), 1)))
        #                batch_accuracy.append(sum(np.argmax(preds.data.cpu().numpy(), 1) == 
        #                                       np.argmax(labels.data.cpu().numpy(), 1)) / float(batch_size))
    accuracy_train.append(train_accuracy/len(dataloaders['train'])
    loss_train.append(train_loss/len(dataloaders['train']))
end = time.time()
print("Training time execution {}".format(end-start))
print("Loss value for training phase: {}".format(train_loss / len(dataloaders['train'])))
print("Accuracy value for training phase: {}".format(train_accuracy / len(dataloaders['train'])))


In [ ]:
epochs = np.arange(1,n_epochs+1)
plt.plot(epochs, loss_train, color='g')
plt.plot(epochs, accuracy_train, color='orange')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training phase')
plt.show() 

In [ ]:
model.eval()
test_loss = 0
test_accuracy = 0 
start = time.time()
for batch_id, (inputs, labels) in enumerate(dataloaders['val']):

    labels = torch.eye(NUM_CLASSES).index_select(dim=0, index=labels)
    inputs, labels = Variable(inputs), Variable(labels)

    if USE_CUDA: inputs, labels = inputs.to(device), labels.to(device)#cuda()

    outputs, masked = model(inputs)
    _,label = torch.max(labels, 1)
    loss = criterion(outputs, label.long())

    test_loss += loss.data[0]
    test_accuracy += (sum(np.argmax(outputs.data.cpu().numpy(), 1) == np.argmax(labels.data.cpu().numpy(), 1)) / float(batch_size))

    if batch_id % 100 == 0:
        print("test accuracy:", sum(np.argmax(outputs.data.cpu().numpy(), 1) == 
                               np.argmax(labels.data.cpu().numpy(), 1)) / float(batch_size))
end = time.time()
print("Test time execution {}".format(end-start))
print("Loss value for test phase: {}".format(test_loss /  len(dataloaders['val']))) 
print("Accuracy value for test phase: {}".format(test_accuracy /  len(dataloaders['val'])))
torch.cuda.empty_cache()